In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2024-05-22 18:43:34.904324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 18:43:34.904420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 18:43:35.034361: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# load data frames
df_train = pd.read_csv('/kaggle/input/dataframes-bal/train_bal.csv')
df_val = pd.read_csv('/kaggle/input/dataframes-bal/val_bal.csv')
df_test = pd.read_csv('/kaggle/input/dataframes-bal/test_bal.csv')

# load joint embeddings
#with open('/kaggle/input/v-embeds/v_embeds_train.pkl', 'rb') as f:
   # v_embeds_train = pickle.load(f) 
    
#with open('/kaggle/input/v-embeds/v_embeds_val.pkl', 'rb') as f:
  #  v_embeds_val = pickle.load(f) 
    
with open('/kaggle/input/j-embeds-bal/j_embeds_test (2).pkl', 'rb') as f:
    j_embeds_test = pickle.load(f) 

In [6]:
print(len(j_embeds_test)) # Liste aus Tensors
print(j_embeds_test[0].shape)
print(type(j_embeds_test[0]))
print(j_embeds_test[0].device)

89
torch.Size([612, 768])
<class 'torch.Tensor'>
cuda:0


In [8]:
X_train = torch.stack(j_embeds_test[:70], dim=0).cpu().numpy()
X_val = torch.stack(j_embeds_test[70:80], dim=0).cpu().numpy()
X_test = torch.stack(j_embeds_test[80:89], dim=0).cpu().numpy()
print(X_train.shape, X_val.shape, X_test.shape)

(70, 612, 768) (10, 612, 768) (9, 612, 768)


In [10]:
# convert labels to numerical values
label_encoder = LabelEncoder()
labels = df_test['AUTHOR'].tolist()
y_train = labels[:70]
y_val = labels[70:80]
y_test = labels[80:89]

y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)
print(y_test)

[9 5 1 3 8 4 6 9 9]


In [16]:
# Define the model
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(612, 768)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val)) # decrease epochs if needed

# Evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')



Epoch 1/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 1s 2s/step - accuracy: 0.0625 - loss: 2.4814

W0000 00:00:1716403778.477292     166 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 893ms/step - accuracy: 0.0589 - loss: 7.5744 - val_accuracy: 0.1000 - val_loss: 59.6968
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1664 - loss: 44.5007 - val_accuracy: 0.3000 - val_loss: 52.9879
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2149 - loss: 48.4140 - val_accuracy: 0.1000 - val_loss: 59.6526
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.2539 - loss: 37.4835 - val_accuracy: 0.0000e+00 - val_loss: 38.4639
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4801 - loss: 21.5182 - val_accuracy: 0.1000 - val_loss: 51.7227
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4844 - loss: 17.3389 - val_accuracy: 0.3000 - val_loss: 35.0666
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6464 - loss: 9.7128 - val_accuracy: 0.1000 - val_loss: 30.5118
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6759 - loss: 6.7566 - val_accuracy: 0.2000 - val_loss: 2

Besser als mit visuellen Features und über Chance Performance. Allerdings nur mit 109 Datensätzen trainiert. Wie kann man sichergehen, dass das Model nicht nur die Textfeatures benutzt? 